In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

In [7]:
df = pd.read_csv('combined_scrape_2.csv')

In [8]:
df.shape

(3543710, 4)

In [9]:
df.head()

,Comment Text,Author,Subreddit,Post ID
0,"Just curious, but what triggered your feelings...",PDXorax,YangForPresidentHQ,ew5ohva
1,Christmas time put up a light-up Yang sign!,PDXorax,YangForPresidentHQ,ew5o8vv
2,"During the general election, get in contact wi...",PDXorax,YangForPresidentHQ,ew42bdx
3,"We need to understand as a country, that the m...",PDXorax,YangForPresidentHQ,evva7l8
4,"Part of the reason for this, is because we nee...",PDXorax,YangForPresidentHQ,evoo179


In [10]:
df_target_train = pd.read_csv('y_yangorno_train.csv')

In [11]:
df_target_train.shape

(2449, 2)

In [12]:
df_target_test = pd.read_csv('y_yangorno_test.csv')

In [13]:
df_target_test.shape

(817, 2)

In [14]:
df_target = pd.concat([df_target_test,df_target_train])

In [15]:
df_target.shape

(3266, 2)

In [16]:
users = list(df_target['Unnamed: 0'])

In [17]:
def clean_df(row):
    if row['Author'] in users:
        return '1'
    else:
        return '0'


In [18]:
df['keep'] = df.apply(clean_df,axis=1)

In [19]:
df = df[df['keep']=='1']

In [20]:
sum_series = df.groupby(['Author','Subreddit']).size()

In [21]:
#exclude out subs from first model
list_of_subs_mod = ['neoliberal','movies','atheism','MMA','Libertarian','teenagers',
    'television','Futurology','samharris','hockey','JoeRogan','exmormon',
    'teslamotors','marvelstudios','hiphopheads','survivor','rupaulsdragrace','Music','magicTCG',
    'vegan','trees','anime','environment','space','keto','askgaybros','StarWars','Android',
    'Christianity','formula1','aznidentity','books','JordanPeterson','gratefuldead','PS4',
    'BasicIncome','cars','Tinder','liberalgunowners','Cricket',
    'NASCAR','TheLastAirbender','HomeImprovement','Fitness','Fuckthealtright',
    'ENLIGHTENEDCENTRISM','LateStageCapitalism','ABoringDystopia','Conservative','tuesday','CryptoCurrency','Bitcoin',
    'nfl','CFB','CHIBears','stupidpol',
    'buildapcsales','Amd','soccer','MLS','Games','gaming']



In [22]:
features = {}
for user in users:
    feature_dict = {}
    for sub in sum_series[user].index:
        if sub in list_of_subs_mod:
            feature_dict[sub]=sum_series[user][sub]
    if len(feature_dict)!=0:
        features[user]=feature_dict


In [23]:
df_feature = pd.DataFrame(features).T

In [24]:
df_feature.head()

,ABoringDystopia,Amd,Android,BasicIncome,Bitcoin,CFB,CHIBears,Christianity,Conservative,Cricket,...,soccer,space,stupidpol,survivor,teenagers,television,teslamotors,trees,tuesday,vegan
Fair_enough42,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN,10.0,NaN,...,1.0,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN
SkepticalJohn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
mwheele86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SgtRockyWalrus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
QuarantineTheHumans,5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0


In [25]:
df_feature.shape

(2718, 62)

In [26]:
df_feature.fillna(0,inplace=True)

In [27]:
#add code to combine like minded subs
df_feature['anti_alt_right_comb']=df_feature['ENLIGHTENEDCENTRISM']+df_feature['Fuckthealtright']
df_feature['bad_capitalism_comb']=df_feature['LateStageCapitalism']+df_feature['ABoringDystopia']
df_feature['conservative_comb']=df_feature['Conservative']+df_feature['tuesday']
df_feature['crypto_comb']=df_feature['CryptoCurrency']+df_feature['Bitcoin']
df_feature['football_comb']=df_feature['nfl']+df_feature['CFB']+df_feature['CHIBears']
df_feature['soccer_comb']=df_feature['soccer']+df_feature['MLS']
df_feature['gaming_comb']=df_feature['gaming']+df_feature['Games']

In [28]:
#remove above single subs
df_feature.drop(['ENLIGHTENEDCENTRISM','Fuckthealtright','LateStageCapitalism','ABoringDystopia','Conservative','tuesday',
                 'CryptoCurrency','Bitcoin','nfl','CFB','CHIBears','soccer','MLS','buildapcsales','Amd'],axis=1,inplace=True)

In [29]:
df_feature = df_feature.div(df_feature.sum(axis=1), axis=0)

In [30]:
df_feature.head(20)

,Android,BasicIncome,Christianity,Cricket,Fitness,Futurology,Games,HomeImprovement,JoeRogan,JordanPeterson,...,teslamotors,trees,vegan,anti_alt_right_comb,bad_capitalism_comb,conservative_comb,crypto_comb,football_comb,soccer_comb,gaming_comb
Fair_enough42,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.094737,0.000000,...,0.010526,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.452632,0.010526,0.000000
SkepticalJohn,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000
mwheele86,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SgtRockyWalrus,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.156250,0.000000,0.000000,0.062500,0.000000,0.000000,0.593750,0.000000,0.031250
QuarantineTheHumans,0.000000,0.000000,0.021277,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.085106,...,0.000000,0.000000,0.053191,0.095745,0.191489,0.063830,0.000000,0.000000,0.000000,0.000000
timelighter,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.022472,0.0,0.000000,0.000000,...,0.000000,0.033708,0.000000,0.000000,0.000000,0.000000,0.000000,0.011236,0.000000,0.112360
JoeChagan,0.039216,0.019608,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.039216,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.392157
dmantzoor,0.000000,0.000000,0.000000,0.000000,0.043478,0.0000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.028986,0.000000,0.000000,0.000000,0.101449,0.000000,0.000000,0.000000,0.000000
PeridotBestGem,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010363,0.000000,0.000000,0.502591,0.005181,0.000000
bantha-fodder12,0.000000,0.000000,0.000000,0.000000,0.000000,0.1875,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.187500,0.125000,0.000000,0.000000,0.062500


In [31]:
df_feature.isnull().sum().head()

Android         3
BasicIncome     3
Christianity    3
Cricket         3
Fitness         3
dtype: int64

In [32]:
df_feature[df_feature['Android'].isnull()]
# X_train[X_train['Android'].isnull()]

,Android,BasicIncome,Christianity,Cricket,Fitness,Futurology,Games,HomeImprovement,JoeRogan,JordanPeterson,...,teslamotors,trees,vegan,anti_alt_right_comb,bad_capitalism_comb,conservative_comb,crypto_comb,football_comb,soccer_comb,gaming_comb
lokikaraoke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RandomJerk2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_left,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_feature = df_feature.drop(index=['pasak1987','C_left','lokikaraoke','RandomJerk2012'],axis=0)

In [34]:
df_target_train = df_target_train.set_index('Unnamed: 0')

In [35]:
df_target_test = df_target_test.set_index('Unnamed: 0')

In [36]:
df_feature.shape

(2714, 54)

In [37]:
df_train = pd.concat([df_feature,df_target_train],axis=1,join='inner')

In [38]:
df_test = pd.concat([df_feature,df_target_test],axis=1,join='inner')

In [39]:
df_train.shape

(2021, 55)

In [40]:
df_test.shape

(693, 55)

In [42]:
df_train.head()

,Android,BasicIncome,Christianity,Cricket,Fitness,Futurology,Games,HomeImprovement,JoeRogan,JordanPeterson,...,trees,vegan,anti_alt_right_comb,bad_capitalism_comb,conservative_comb,crypto_comb,football_comb,soccer_comb,gaming_comb,target
Horcruxno12,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.032258,0.000000,0
squeegee1234,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.333333,0.0,0.000000,0.000000,0.000000,0
WolverineSanders,0.0,0.0,0.0,0.0,0.0,0.366667,0.0,0.033333,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.033333,0
ZgylthZ,0.0,0.0,0.0,0.0,0.0,0.872340,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.042553,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0
JoshPastnerIsMyDad,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.196154,0.734615,0.000000,0


In [43]:
df_test.head()

,Android,BasicIncome,Christianity,Cricket,Fitness,Futurology,Games,HomeImprovement,JoeRogan,JordanPeterson,...,trees,vegan,anti_alt_right_comb,bad_capitalism_comb,conservative_comb,crypto_comb,football_comb,soccer_comb,gaming_comb,target
Fair_enough42,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.094737,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.105263,0.0,0.452632,0.010526,0.00000,1
SkepticalJohn,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.030303,0.000000,0.000000,0.030303,0.000000,0.0,0.000000,0.000000,0.00000,0
mwheele86,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,1
SgtRockyWalrus,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.156250,0.000000,0.000000,0.062500,0.000000,0.0,0.593750,0.000000,0.03125,0
QuarantineTheHumans,0.0,0.0,0.021277,0.0,0.0,0.0,0.0,0.0,0.000000,0.085106,...,0.000000,0.053191,0.095745,0.191489,0.063830,0.0,0.000000,0.000000,0.00000,0


In [44]:
df_train.to_csv('yangorno_train.csv')

In [45]:
df_test.to_csv('yangorno_test.csv')

In [36]:
#modeling

In [37]:
lr = LogisticRegression()

In [38]:
smt = SMOTE()

In [39]:
X_train = df_train.drop('target',axis=1)

In [40]:
y_train = df_train['target']

In [41]:
X_train_2,y_train_2 = smt.fit_sample(X_train,y_train)

In [42]:
X_test = df_test.drop('target',axis=1)

In [43]:
y_test = df_test['target']

In [44]:
lr.fit(X_train,y_train)

/Users/yichenh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
lr.score(X_train,y_train)

0.7828458106098165

In [46]:
y_train.value_counts(normalize=True)

0    0.754586
1    0.245414
Name: target, dtype: float64

In [47]:
lr.score(X_test,y_test)

0.7862266857962698

In [48]:
y_test.value_counts(normalize=True)

0    0.756098
1    0.243902
Name: target, dtype: float64

In [49]:
pd.Series(lr.predict(X_test)).value_counts(normalize=True)

0    0.92109
1    0.07891
dtype: float64

In [50]:
lr2 = LogisticRegression()

In [51]:
lr2.fit(X_train_2,y_train_2)

/Users/yichenh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [52]:
lr2.score(X_train_2,y_train_2)

0.7296320630749015

In [53]:
lr2.score(X_test,y_test)

0.6642754662840746

In [54]:
y_test.value_counts(normalize=True)

0    0.756098
1    0.243902
Name: target, dtype: float64

In [55]:
pd.Series(lr2.predict(X_test)).value_counts(normalize=True)

0    0.58967
1    0.41033
dtype: float64

In [56]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [57]:
confusion_matrix(lr.predict(X_test),y_test)

array([[510, 132],
       [ 17,  38]])

In [58]:
confusion_matrix(lr2.predict(X_test),y_test)

array([[352,  59],
       [175, 111]])

In [59]:
y_test.value_counts()

0    527
1    170
Name: target, dtype: int64

In [60]:
lr3 = LogisticRegression(class_weight='balanced')

In [61]:
lr3.fit(X_train,y_train)

/Users/yichenh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
lr3.score(X_test,y_test)

0.6614060258249641

In [63]:
confusion_matrix(lr3.predict(X_test),y_test)

array([[352,  61],
       [175, 109]])

In [67]:
# results2 = pd.DataFrame(lr2.coef_.T,X_test.columns)

In [120]:
# results = pd.DataFrame(lr.coef_.T,X_test.columns)